# Regex Labeling

## Data Loading

In [1]:
import pandas as pd
import numpy as np
import re
import csv
import json

In [2]:
GRAPH_VER = 6

DATASET_PATH = './data/code_blocks_clean.csv' ## CODE_COLUMN = 'code_block'
OUTPUT_DATASET_PATH = './data/code_blocks_regex_graph_v{}.csv'.format(GRAPH_VER)
CODE_COLUMN = 'code_block'
GRAPH_DIR = './graph/graph_v{}.txt'.format(GRAPH_VER)

In [3]:
df = pd.read_csv(DATASET_PATH, encoding='utf-8', sep='\t')#, error_bad_lines=False, sep=',')#quoting=csv.QUOTE_NONE,

In [4]:
# df = df[[CODE_COLUMN, 'tag']]

In [5]:
print(df.isna().sum())
df = df.dropna(subset=[CODE_COLUMN]).reset_index()

code_block      1
tag           488
dtype: int64


In [6]:
# def wordListToFreqDict(wordlist):
#     def sortFreqDict(freqdict):
#         aux = [(freqdict[key], key) for key in freqdict]
#         aux.sort()
#         aux.reverse()
#         return aux
#     wordfreq = [wordlist.count(p) for p in wordlist]
#     return sortFreqDict(dict(list(zip(wordlist,wordfreq))))

In [7]:
# tokens = (" ".join(" ".join(df['Code'].to_list()).split('\n')).split('.'))#.split('')
# wordListToFreqDict(tokens)

In [8]:
# %%time
# wordListToFreqDict(df.code.to_string())

In [9]:
# %%time
# nl2ml = pd.read_csv(KK_PATH + 'nl2ml.csv')
# nl2ml = nl2ml.rename({'':'code_block', '':'method_tag'})
# nl2ml_vis = nl2ml[nl2ml['method_tag'] == 'Visualization']
# tokens_visualization = wordListToFreqDict(nl2ml_vis.code_block.to_string())

In [10]:
def tokens_search(df, tokens, new_column_name):
    df[new_column_name] = 0
    for i in range(len(df)):
        percents = str(round(100*i/len(df),1))
        print(percents + '%\r', end='')
        row = df[CODE_COLUMN][i]
        for token in tokens:
            result = re.search(token.replace('(','\('), row)
            if result!=None:
                df[new_column_name][i] = 1
                break
    return df

## Regex Labels

In [15]:
with open(GRAPH_DIR, "r") as graph_file:
    graph = json.load(graph_file)
    # graph = graph_file.read()

JSONDecodeError: Expecting ',' delimiter: line 4 column 41 (char 204)

In [ ]:
%%time
for i in range(0, len(graph)):
    vertice = list(graph.keys())[i]
    print('\n' + vertice)
    tokens = graph[vertice]
    df = tokens_search(df, tokens, vertice)

In [ ]:
df['preprocessing'].value_counts()

In [ ]:
df.to_csv(OUTPUT_DATASET_PATH, index=False)

In [ ]:
# for col in list(graph.keys()):
#     display(df[df['Data format'] == 'Table'][col].value_counts())

In [ ]:
print('finished')

## Regex Evaluation

In [53]:
from sklearn.metrics import f1_score, precision_score, recall_score
import dagshub

In [16]:
GRAPH_VER = 5
REGEXED_DATA_PATH = "./data/golden_884_set.csv"
TAGS = ['import', 'data_import', 'data_export', 'preprocessing',
                    'visualization', 'model', 'deep_learning_model', 'train', 'predict']
REGEX_TAGS = [el+'_regex_v{}'.format(GRAPH_VER) for el in TAGS]
regexed_data = pd.read_csv(REGEXED_DATA_PATH)
regexed_data.head()

,code_block,import,data_import,data_export,preprocessing,visualization,model,deep_learning_model,train,predict,import_regex,data_import_regex_v5,data_export_regex_v5,preprocessing_regex_v5,visualization_regex_v5,model_regex_v5,deep_learning_model_regex_v5,train_regex_v5,predict_regex_v5
0,\nprint(f()) \n\nx = 5\ndef f():\n y = 2...,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,"\nprint(int('807') + 1),\n\nimport numpy as np...",1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,\nimport numpy as np \nimport pandas as pd \ni...,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,\n Average Precision-Recall Score ={0:0.2f}'.f...,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0
4,"\nplot_confusion_matrix(actual_cm, labels, tit...",1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0


In [25]:
Y_test, Y_pred = regexed_data[TAGS], regexed_data[REGEX_TAGS]

In [60]:
base_f1 = f1_score(Y_test, Y_pred, average='weighted')
base_precision = precision_score(Y_test, Y_pred, average='weighted')
base_recall = recall_score(Y_test, Y_pred, average='weighted')
regex_results = {'test_f1_score': base_f1
                , 'test_precision': base_precision
                , 'test_recall': base_recall}
regex_results

{'test_f1_score': 0.7568760919796619,
 'test_precision': 0.9258695652173914,
 'test_recall': 0.6784420289855072}

In [58]:
for i, tag in enumerate(TAGS):
    tag_results = (round(f1_score(Y_test.iloc[:, i], Y_pred.iloc[:, i], average='weighted'),4),\
                    round(precision_score(Y_test.iloc[:, i], Y_pred.iloc[:, i], average='weighted'),4),\
                    round(recall_score(Y_test.iloc[:, i], Y_pred.iloc[:, i], average='weighted'),4))
    print(tag)
    print(tag_results)
    print('------')

import
(1.0, 1.0, 1.0)
------
data_import
(0.9767, 0.9772, 0.9774)
------
data_export
(0.9931, 0.9933, 0.9932)
------
preprocessing
(0.8392, 0.8781, 0.8597)
------
visualization
(0.8773, 0.9047, 0.8914)
------
model
(0.9127, 0.9258, 0.9208)
------
deep_learning_model
(0.918, 0.9516, 0.8948)
------
train
(0.9683, 0.9683, 0.9683)
------
predict
(0.9425, 0.9463, 0.94)
------


In [63]:
data_meta = {'DATASET_PATH': REGEXED_DATA_PATH
            ,'nrows': regexed_data.shape[0]
            ,'graph_ver': GRAPH_VER
            ,'label': TAGS
            ,'model': 'regex_v{}'.format(GRAPH_VER)
            ,'script_dir': './regex.ipynb'
            ,'task': 'regex evaluation'}

with dagshub.dagshub_logger() as logger:
    logger.log_hyperparams(data_meta)
    logger.log_metrics(regex_results)